In [14]:
import json
import numpy as np
from scipy.stats import norm
from scipy.special import logsumexp

def test_supervised_Normal(X, nsample, a, rho, Tau, Lambda, T, w = None):
    '''
    Given the estimated model, return the retimated labels and the posterior distribution of labels.
    res["labels"]: the list of estmate labels
    res["loglikelihood"]: the list of the posterior distribution.
    '''
    nlabel = len(T)
    N = len(X)
    d = len(X[0])
    if not w: # if no info, assume uniform
        w = np.ones(nlabel)


    probs = []
    for j in range(N):
        tem = []

        for y in range(nlabel):
            gs = np.random.dirichlet(a * rho, nsample)
            us = np.dot(gs, T[y].T)
            lambdas = np.dot(us, Lambda)
            taus = np.dot(us, Tau)
            sigmas = np.sqrt(1 / lambdas)
            mus = taus / lambdas

            xs = np.tile(X[j,], (nsample, 1))
            likelihoods = norm.pdf(xs, mus, sigmas)
            loglikelihood = np.log(w[y]) + logsumexp(np.log(likelihoods).sum(axis = 1)) # since we use the same nsample, no need to subtract log(nsample)

            tem.append(loglikelihood)

        probs.append(tem)

    labels = []
    for i in range(N):
        post_prob = probs[i]
        label = post_prob.index(max(post_prob))
        labels.append(label)
    
    return {"labels": labels, "loglikelihood": probs}

def document_generator_Normal(a, rho, T, Lambda, Tau, N, w = None):
    '''
    a, rho: corpus-level parameters
    T: transformation matrix. ntopic * K * dg
    Lambda, Tau: topics. K*d matrix. Lambda are positive.
    N: the number of documents.
    w: the probability of Y.
    
    Lambda = 1/sigma^2
    Tau = mu/sigma^2
    or
    sigma = 1/Lambda
    mu = Tau/Lambda

    x|u ~ normal(lambda_x, tau_x), where lambda_x = sum(u_i * lambda_i) and tau_x = sum(u_i * tau_i)
    
    output: 
    X: N*d, X[i] = document[i]
    Y: Y[i] = label[i]
    G: membership
    U: transformed membership
    '''

    nlabel = len(T) # number of classes
    d = len(Tau[0]) # dim(x)
    
    Y = np.random.choice(list(range(nlabel)), size = N, p = w) # labels

        
    G = np.random.dirichlet(a * rho, N)
    U = np.array([np.dot(T[Y[i]], G[i]) for i in range(N)])

    X = np.zeros((N, d))
    for i in range(N):
        u = U[i]
        for j in range(d):
            lambdax = np.dot(u, Lambda[:, j])
            taux = np.dot(u, Tau[:, j])
            sx = 1 / lambdax
            mux = sx * taux
            X[i][j] = np.random.normal(mux, np.sqrt(sx))

    return X, Y, G, U

In [7]:
# read parameters
id = 1
k0 = 2
k1 = 3
nlabel = 3
d = 20
mu_Mu, sigma2_Mu, alpha_Lambda, beta_Lambda = 0, 10, 2, 4

Ntrain = 200
Ntest = 100
nchain = 2
ntrace = 2000
nskip = 2


# generate  parameters
np.random.seed(1)
dg = k0 + k1 
K = ntopic = nlabel * k0 + k1

b = 0.1
alpha = np.ones(dg)

# define T
T = []
for i in range(nlabel):
    tem = np.block([
        [np.zeros((k0 * i, k0 + k1))],
        [np.eye(k0), np.zeros((k0, k1))],
        [np.zeros((k0 * (nlabel - i - 1), k0 + k1))],
        [np.zeros((k1, k0)), np.eye(k1)]
    ])
    T.append(tem)

rho = np.random.dirichlet(alpha, 1)[0]
a = np.random.exponential(1 / b, 1)[0]
Lambda = np.random.gamma(shape = alpha_Lambda, scale = 1 / beta_Lambda, size = (K, d))
Mu = np.random.normal(mu_Mu, np.sqrt(sigma2_Mu / Lambda), (K, d))
Tau = Mu * Lambda
# generate dataset, split for CV
X, Y, G, U = document_generator_Normal(a, rho, T, Lambda, Tau, Ntrain)
training_set = {'X': X, "Y": Y, "G": G, "U":U}
X, Y, G, U = document_generator_Normal(a, rho, T, Lambda, Tau, Ntest)
test_set = {'X': X, "Y": Y, "G": G, "U":U}

In [24]:
with open("test_supervised_train.txt", "r") as f:
    res = json.loads(f.read())

In [25]:
a1 = res["a"]
rho1 = np.array(res["rho"])
Mu1 = np.array(res["Mu"])
Lambda1 = np.array(res["Lambda"])



In [27]:
pre1 = test_supervised_Normal(test_set["X"], 1000, a, rho, Tau, Lambda, T, w = None)
sum(np.array(pre1["labels"]) == test_set["Y"])

97

In [26]:
Tau1 = Mu1 * Lambda1
pre1 = test_supervised_Normal(test_set["X"], 1000, a1, rho1, Tau1, Lambda1, T, w = None)
sum(np.array(pre1["labels"]) == test_set["Y"])

93

In [28]:
Tau1 = Mu1 * Lambda1
pre1 = test_supervised_Normal(training_set["X"], 1000, a1, rho1, Tau1, Lambda1, T, w = None)
sum(np.array(pre1["labels"]) == training_set["Y"])

191